# LNDL v2: Live API Testing

This notebook demonstrates LNDL (Lion's Natural Directive Language) v2 with live API calls.

**Goals:**
1. Test LNDL parsing with real model responses
2. Validate structured output generation
3. Demonstrate the Operative pattern with actions
4. Test streaming output

**Requirements:**
- Set `ANTHROPIC_API_KEY` or `OPENAI_API_KEY` environment variable
- lionpride package installed

In [ ]:
import os
import sys

# Add src to path for development
sys.path.insert(0, os.path.abspath("../src"))

# Load environment variables from .env file
from dotenv import load_dotenv

load_dotenv()

# Check for API keys
has_anthropic = bool(os.environ.get("ANTHROPIC_API_KEY"))
has_openai = bool(os.environ.get("OPENAI_API_KEY"))

print(f"Anthropic API available: {has_anthropic}")
print(f"OpenAI API available: {has_openai}")

if not has_anthropic and not has_openai:
    print("\nWarning: No API key found. Set ANTHROPIC_API_KEY or OPENAI_API_KEY to run live tests.")

## 1. Basic Setup

Import core components and create a session.

In [ ]:
from pydantic import BaseModel, Field

from lionpride.lndl import parse_lndl_fuzzy
from lionpride.operations import create_action_operative, create_operative_from_model
from lionpride.operations.lndl import generate_lndl_spec_format, prepare_lndl_messages
from lionpride.operations.validation import validate_response
from lionpride.services import iModel
from lionpride.services.providers.oai_chat import OAIChatEndpoint
from lionpride.session import Session

# Create session
session = Session()
branch = session.create_branch()

print(f"Session created: {session.id}")
print(f"Branch created: {branch.id}")

## 2. Define Response Models

Define Pydantic models for structured outputs.

In [ ]:
class AnalysisResult(BaseModel):
    """Structured analysis result."""

    topic: str = Field(..., description="Topic analyzed")
    summary: str = Field(..., description="Brief summary of findings")
    key_points: list[str] = Field(default_factory=list, description="Key points")
    confidence: float = Field(..., ge=0.0, le=1.0, description="Confidence score")


class CodeReview(BaseModel):
    """Code review result."""

    file_path: str = Field(..., description="File being reviewed")
    issues: list[str] = Field(default_factory=list, description="Issues found")
    suggestions: list[str] = Field(default_factory=list, description="Improvement suggestions")
    quality_score: float = Field(..., ge=0.0, le=10.0, description="Code quality score")
    approve: bool = Field(..., description="Whether to approve")


# Create operatives
analysis_operative = create_operative_from_model(AnalysisResult, name="Analysis")
review_operative = create_operative_from_model(CodeReview, name="CodeReview")

print("Analysis Operative:")
print(f"  - Operable: {analysis_operative.operable}")
print(f"  - Specs: {[s.name for s in analysis_operative.operable.get_specs()]}")

## 3. Generate LNDL Prompt

See the LNDL format specification that guides the model.

In [ ]:
lndl_prompt = generate_lndl_spec_format(analysis_operative)
print("LNDL Spec Format:")
print("-" * 60)
print(lndl_prompt)

## 4. Parse LNDL Response (Mock)

Test LNDL parsing with a mock response.

In [ ]:
# Simulate model response in LNDL format
mock_lndl_response = """<lvar AnalysisResult.topic t>Machine Learning Fundamentals</lvar>
<lvar AnalysisResult.summary s>An overview of core ML concepts including supervised learning, neural networks, and optimization techniques.</lvar>
<lvar AnalysisResult.key_points kp>["Supervised vs unsupervised learning", "Neural network architectures", "Gradient descent optimization", "Model evaluation metrics"]</lvar>
<lvar AnalysisResult.confidence c>0.92</lvar>
OUT{analysis: [t, s, kp, c]}"""

print("Mock LNDL Response:")
print(mock_lndl_response)
print()

# Parse the LNDL
parsed = parse_lndl_fuzzy(mock_lndl_response, analysis_operative.operable)
print("Parsed Result:")
print(f"  Topic: {parsed.fields.get('analysis').topic}")
print(f"  Summary: {parsed.fields.get('analysis').summary}")
print(f"  Key Points: {parsed.fields.get('analysis').key_points}")
print(f"  Confidence: {parsed.fields.get('analysis').confidence}")

## 5. Validation Strategy

Test the validation strategy pattern.

In [ ]:
# Test validation with different responses

# Valid LNDL response
valid_lndl = """<lvar AnalysisResult.topic t>Python Best Practices</lvar>
<lvar AnalysisResult.summary s>Guidelines for writing clean Python code.</lvar>
<lvar AnalysisResult.key_points kp>["Use type hints", "Write docstrings"]</lvar>
<lvar AnalysisResult.confidence c>0.85</lvar>
OUT{analysis: [t, s, kp, c]}"""

result = validate_response(
    valid_lndl,
    operable=analysis_operative,
    threshold=0.6,
)

print(f"Validation Success: {result.success}")
if result.success:
    print(f"Data: {result.data}")
else:
    print(f"Error: {result.error}")

## 6. Action Operative

Create an operative with tool/action support.

In [ ]:
# Create operative with actions
action_operative = create_action_operative(
    base_model=AnalysisResult,
    reason=True,
    actions=True,
    name="AnalysisWithActions",
)

print("Action Operative Specs:")
for spec in action_operative.operable.get_specs():
    print(f"  - {spec.name}: {spec.base_type}")

print(f"\nSupports Actions: {action_operative._supports_actions}")
print(f"Request Exclude: {action_operative.request_exclude}")

## 7. Live API Test (Optional)

Test with real API if key is available.

In [ ]:
# Skip if no API key
if not has_anthropic and not has_openai:
    print("Skipping live API test - no API key available")
else:
    # Configure endpoint based on available API key
    if has_openai:
        endpoint = OAIChatEndpoint(model="gpt-4o-mini")
        model = iModel(backend=endpoint, name="openai_chat")
    else:
        # For Anthropic, would need AnthropicEndpoint (not shown here)
        print("Anthropic endpoint not configured in this demo")
        model = None

    if model:
        session.services.register(model)
        print(f"Registered model: {model.name}")

In [ ]:
# Live test with LNDL format
if has_openai and model:
    from lionpride.operations.lndl import generate_lndl_spec_format, prepare_lndl_messages
    from lionpride.session.messages import InstructionContent, Message

    # Create operative for LNDL
    operative = create_operative_from_model(AnalysisResult, name="Analysis")

    # Show the LNDL spec format
    print("=== LNDL Spec Format ===")
    print(generate_lndl_spec_format(operative))
    print("-" * 60)

    # IMPORTANT: Don't pass response_model to InstructionContent for LNDL mode
    # The LNDL spec is in the system message, not the user message
    ins_content = InstructionContent.create(
        instruction="Analyze the topic 'Python async/await patterns'. Provide a structured analysis.",
        # NO response_model - LNDL format guidance is in system message
    )
    ins_msg = Message(content=ins_content, sender="user", recipient="assistant")

    # Prepare LNDL messages (injects LNDL system prompt + spec)
    messages = prepare_lndl_messages(
        session=session,
        branch=branch,
        ins_msg=ins_msg,
        operable=operative.operable,
    )

    # Invoke the model
    print("\nInvoking model...")
    calling = await model.invoke(messages=messages, max_tokens=1000)
    response_text = calling.execution.response.data

    print("\n=== Model Response ===")
    print(response_text)
    print("-" * 60)

    # Parse LNDL response
    print("\n=== Parsing LNDL ===")
    try:
        parsed = parse_lndl_fuzzy(response_text, operative.operable)
        analysis = parsed.fields.get("analysis")

        print(f"✓ Topic: {analysis.topic}")
        print(f"✓ Summary: {analysis.summary}")
        print(f"✓ Key Points: {analysis.key_points}")
        print(f"✓ Confidence: {analysis.confidence}")
    except Exception as e:
        print(f"✗ Parse error: {e}")
else:
    print("Skipping live API test - no OpenAI API key or model not configured")

## 8. Streaming Test (Optional)

Test streaming output.

In [ ]:
# Streaming test using StreamChannel (handles SSE parsing automatically)
if has_openai and model:
    print("Streaming response via StreamChannel:")
    print("-" * 40)

    # invoke_stream_with_channel wraps invoke_stream in StreamChannel
    # which automatically parses SSE and extracts content
    channel = await model.invoke_stream_with_channel(
        messages=[{"role": "user", "content": "Count from 1 to 5, one number per line."}],
        max_tokens=50,
    )

    # Add a consumer that prints each chunk as it arrives
    channel.add_consumer(
        lambda chunk: print(chunk.content, end="", flush=True) if not chunk.is_final else None
    )

    # Iterate through the channel (this drives the stream)
    async for _chunk in channel:
        pass  # Consumer handles printing

    print("\n" + "-" * 40)
    print(f"Full accumulated text: {channel.get_accumulated()}")
    print(f"Total chunks received: {len(channel.get_buffer())}")
else:
    print("Skipping streaming test - no OpenAI API key or model not configured")

## 9. Error Handling

Test LNDL error cases.

In [ ]:
from lionpride.lndl.errors import MissingOutBlockError, UnresolvedAliasError

# Test missing OUT block
malformed_lndl = """<lvar AnalysisResult.topic t>Test</lvar>
<lvar AnalysisResult.summary s>Summary</lvar>"""

try:
    parse_lndl_fuzzy(malformed_lndl, analysis_operative.operable)
except MissingOutBlockError as e:
    print(f"Expected error caught: {e}")

# Test unresolved alias
bad_alias_lndl = """<lvar AnalysisResult.topic t>Test</lvar>
OUT{analysis: [t, unknown_alias]}"""

try:
    parse_lndl_fuzzy(bad_alias_lndl, analysis_operative.operable)
except UnresolvedAliasError as e:
    print(f"Expected error caught: {e}")

<cell_type>markdown</cell_type>## 10. Summary

LNDL v2 features demonstrated:

1. **Operative Pattern** - Wrap Pydantic models with validation support
2. **LNDL Format Generation** - `operations/lndl/formatting.py` generates spec format
3. **Message Preparation** - `prepare_lndl_messages()` composes system + user messages
4. **Fuzzy Parsing** - Flexible parsing with alias resolution in `lndl/` module
5. **Validation Strategy** - Configurable validation with thresholds
6. **Action Support** - Tool/action integration for agentic workflows
7. **Streaming** - Async streaming with channel abstraction
8. **Error Handling** - Typed errors for debugging

**Architecture:**
- `InstructionContent` handles JSON format (with response_model)
- `operations/lndl/` handles LNDL format (via prepare_lndl_messages)
- `lndl/` module handles core parsing (parse_lndl_fuzzy)

In [ ]:
print("LNDL v2 Demo Complete!")